# TensorFlow 测试样例

In [13]:
import tensorflow as tf
a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")
ADD = tf.add(a, b)
sess = tf.Session()
result = sess.run(ADD)
print(result)

[3. 5.]


TensorFlow程序一般可以分为两个阶段。在第一个阶段需要定义计算图中所有的计算，第二个阶段为执行计算。
在TensorFlow程序中，系统会自动维护一个默认的计算图，通过tf.get_default_graph函数可以获得当前默认的计算图。

In [14]:
print(a.graph is tf.get_default_graph())

True


除了使用默认的计算图，Tensorflow支持通过tf.Graph函数来生成新的计算图。不同计算图上的张量和运算都不会共享。

In [26]:
# 在不同计算图上定义和使用变量
import tensorflow as tf
g1 = tf.Graph()
with g1.as_default():
    # 在计算图g1中定义变量“v”，并设初始值为0.
    v = tf.get_variable("v", shape=[1], initializer=tf.zeros_initializer())
    print(v)

g2 = tf.Graph()
with g2.as_default():
    # 在计算图g2中定义变量“v”，并设初始值为1.
    v = tf.get_variable("v", shape=[1], initializer=tf.ones_initializer())
    print(v)
    
# 在计算图g1中读取变量“v”的取值
with tf.Session(graph=g1) as sess:
    # 初始化变量节点，并运行
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))
        
# 在计算图g2中读取变量“v”的取值
with tf.Session(graph=g2) as sess:
    # 初始化变量节点，并运行
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))


<tf.Variable 'v:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'v:0' shape=(1,) dtype=float32_ref>
[0.]
[1.]


In [22]:
# 在计算图g1中读取变量“v”的取值
with tf.Session(graph=g1) as sess:
    # 初始化变量节点，并运行
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))
        
# init = tf.global_variables_initializer()
# with tf.Session(graph=g1) as sess:
#     sess.run(init)
#     with tf.variable_scope("", reuse=True):
#         print(sess.run(tf.get_variable("v")))

[0.]


TensorFlow计算图可以通过tf.Graph.device函数来指定运行计算机设备。

In [28]:
g = tf.Graph()
with g.device('/cpu:0'):
    a = tf.constant([1.0, 2.0], name="a")
    b = tf.constant([2.0, 3.0], name="b")
    ADD = tf.add(a, b)
    sess = tf.Session()
    result = sess.run(ADD)
    print(result)

[3. 5.]


In [34]:
import tensorflow as tf
a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")
ADD = tf.add(a, b)
print(ADD)
# 张量的三个属性：名字，维度，类型
# 当类型不同会报错，如：a = tf.constant([1, 2], name="a")
sess = tf.Session()
result = sess.run(ADD)
print(result)

Tensor("Add_6:0", shape=(2,), dtype=float32)
[3. 5.]


In [39]:
import tensorflow as tf
a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")
ADD = tf.add(a, b)
print(ADD)
sess = tf.Session()
# 两种方法具有相同的功能
print(sess.run(ADD))
print(ADD.eval(session=sess))

Tensor("Add_11:0", shape=(2,), dtype=float32)
[3. 5.]
[3. 5.]


Tensor("Add_10:0", shape=(2,), dtype=float32)
[3. 5.]
[3. 5.]
"Add_10:0":说明了ADD这个张量是计算节点"add"输出的第一个结果（编号从0开始）
shape=(2,):说明张量ADD是一个一维数组，这个数组的长度是2
dtype=float32:张量的属性

## get_Variable与Variablede 区别

### Variable的用法

In [1]:
import tensorflow as tf
var1 = tf.Variable(1.0, name='firstvar')
print("var1:", var1.name)
var1 = tf.Variable(2.0, name='firstvar')
print("var1:", var1.name)
var2 = tf.Variable(3.0)
print("var2:", var2.name)
var2 = tf.Variable(4.0)
print("var2:", var2.name)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("Var1=", var1.eval())
    print("var2=", var2.eval())
#     print("Var1=", sess.run(var1))
#     print("var2=", sess.run(var2))

var1: firstvar:0
var1: firstvar_1:0
var2: Variable:0
var2: Variable_1:0
Var1= 2.0
var2= 4.0


### get_Variable的用法

In [2]:
import tensorflow as tf
get_var1 = tf.get_variable("firstvar",[1], initializer=tf.constant_initializer(0.3))
print ("get_var1:",get_var1.name)

get_var1 = tf.get_variable("firstvar",[1], initializer=tf.constant_initializer(0.4))
print ("get_var1:",get_var1.name)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("get_var1=",get_var1.eval())
    
'''
可以看到程序在定义第二个get_var1时发生崩溃了。这表明使用get_variable只能定义一次指定名称的变量。
同时由于变量firstvar在之前用Variable生成过一次，所以系统自动变成了firstvar_2:0。
'''

get_var1: firstvar_2:0


ValueError: Variable firstvar already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-2-cb0a3afdfb31>", line 2, in <module>
    get_var1 = tf.get_variable("firstvar",[1], initializer=tf.constant_initializer(0.3))
  File "/Users/xxy/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/xxy/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2850, in run_ast_nodes
    if self.run_code(code, result):


In [1]:
import tensorflow as tf
var1 = tf.Variable(1.0, name='firstvar')
print("var1:", var1.name)
var1 = tf.Variable(2.0, name='firstvar')
print("var1:", var1.name)
var2 = tf.Variable(3.0)
print("var2:", var2.name)
var2 = tf.Variable(4.0)
print("var2:", var2.name)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("Var1=", var1.eval())
    print("var2=", var2.eval())
get_var1 = tf.get_variable("firstvar",[1], initializer=tf.constant_initializer(0.3))
print ("get_var1:",get_var1.name)

get_var2 = tf.get_variable("firstvar",[1], initializer=tf.constant_initializer(0.4))
print ("get_var1:",get_var1.name)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("get_var1=",get_var1.eval())
'''
仍然定义个get_var1，不同的是他的名字变成了firstvar1，这样就没问题了。
同样，新的get_var1会在图中生效，所以它的输出值是4.0。
'''

var1: firstvar:0
var1: firstvar_1:0
var2: Variable:0
var2: Variable_1:0
Var1= 2.0
var2= 4.0
get_var1: firstvar_2:0
get_var1: firstvar1:0
get_var1= [0.4]


In [1]:
import tensorflow as tf
get_var1 = tf.get_variable("firstvar",[1], initializer=tf.constant_initializer(0.3))
print ("get_var1:",get_var1.name)

get_var1 = tf.get_variable("firstvar2",[1], initializer=tf.constant_initializer(0.4))
print ("get_var1:",get_var1.name)



get_var1: firstvar:0


ValueError: Variable firstvar already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-1-b0218c1a17c1>", line 2, in <module>
    get_var1 = tf.get_variable("firstvar",[1], initializer=tf.constant_initializer(0.3))
  File "/Users/xxy/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/xxy/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2850, in run_ast_nodes
    if self.run_code(code, result):


### 在特定的作用域下获取变量
使用get_variable，以及嵌套variable_scope
由上面的例子知道，使用get_variable创建两个同样名字（name）的变量的（即name相同不行，变量名相同则覆盖）是行不通的。下面的代码会报错，如果真的想这么做，可以使用variable_scope将他们隔开。
```python
var1 = tf.get_variable("firstvar", [2], dtype=tf.float32)
var2 = tf.get_variable("firstvar",[2], dtype=tf.float32)
```

In [4]:
# 隔开
import tensorflow as tf
with tf.variable_scope("test1"):
    var1 = tf.get_variable("firstvar", [2], dtype=tf.float32)
with tf.variable_scope("test2"):
    var2 = tf.get_variable("firstvar", [2], dtype=tf.float32)
print("var1:", var1.name)
print("var2:", var2.name)
'''
var1和var2都使用firstvar的名字来定义，在两个不同的作用域，生成两个不同的变量。
'''

var1: test1/firstvar:0
var2: test2/firstvar:0


'\nvar1和var2都使用firstvar的名字来定义，在两个不同的作用域，生成两个不同的变量。\n'

In [8]:
'''
scope支持嵌套
'''
import tensorflow as tf
tf.reset_default_graph()
with tf.variable_scope("test1"):
    var1 = tf.get_variable("firstvar", [2], dtype=tf.float32)
    with tf.variable_scope("test2"):
        var2 = tf.get_variable("firstvar", [2], dtype=tf.float32)
print("var1:", var1.name)
print("var2:", var2.name)

var1: test1/firstvar:0
var2: test1/test2/firstvar:0


使用作用域中reuse参数实现共享变量的功能。
variable_scope里面有个reuse=True属性，表示使用已经定义过的变量。
此时在使用get_variable将不会创建新的变量，而是去计算图中get_variable所创建的变量中找到与name相同的变量。

In [3]:
import tensorflow as tf
# tf.get_variable在创建变量的时候 ，回去检查图（一个计算任务）中是否已经创建过该变量。
# 如果创建过并且本次调用时没有被设置为共享方式就会报错。
# 调用tf.reset_default_graph()，将图（一个计算任务）里面的变量清空。
tf.reset_default_graph() 
with tf.variable_scope("test1"):
    var1 = tf.get_variable("var1_name", [1], dtype=tf.float32)
    with tf.variable_scope("test2"):
        var2 = tf.get_variable("var1_name", [1], dtype=tf.float32)
    
print(var1.name)
print(var2.name)

with tf.variable_scope("test1", reuse=True):
    var3 = tf.get_variable("var1_name", [1], dtype=tf.float32)
    with tf.variable_scope("test2"):
        var4 = tf.get_variable("var1_name", [1], dtype=tf.float32) 
        
print(var3.name)
print(var4.name)

#通过设置tf.variable_scope的reuse参数，来对作用域下的指定name的变量进行共享。
#var1和var3的输出名字是一样的，var2和var3输出的名字是一样的，这就实现了变量的共享。

test1/var1_name:0
test1/test2/var1_name:0
test1/var1_name:0
test1/test2/var1_name:0


### 初始化共享变量的作用域——继承
演示variable_scope中get_variable初始化的继承功能，以及嵌套variable_scope的继承功能。
variable_scop和get_variable都要初始化功能。在初始化时，如果没有对当前变量初始化，则TensorFlow会默认使用作用域的初始化方法对其初始化，并且作用域的初始化方法也有继承功能

In [11]:
import tensorflow as tf
tf.reset_default_graph()
with tf.variable_scope("test1", initializer=tf.constant_initializer(0.4)): # 将test1作用域进行初始化为4.0
    var1 = tf.get_variable("firstvar", shape=[2], dtype=tf.float32) # var1没有初始化
    with tf.variable_scope("test2"): # 嵌套的test2作用域也没有初始化
        var2 = tf.get_variable("firstvar", shape=[2], dtype=tf.float32) # var1没有初始化
        var3 = tf.get_variable("var3", shape=[2], initializer=tf.constant_initializer(0.3)) # var3进行了初始化
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("var1:", var1.eval())
    print("var2:", var2.eval())
    print("var3:", var3.eval())
# 从结果看，var1数组为0.4，表明继承了test1的中值；
# var2数组值为0.4，表明其所在的作用域也继承了test1的初始化；
# 变量var3在创建时同步指定了初始化操作。所以数组值为0.3

var1: [0.4 0.4]
var2: [0.4 0.4]
var3: [0.3 0.3]


注意：**在多模型训练中，常常会使用variable_scope对模型间张量进行区分。同时，统一为学习参数进行默认的初始化。在变量共享方面，还可以使用tf.AUTO_REUSE来为reuse属性赋值。tf.AUTO_REUSE可以实现第一次调用variable_scope时，传入的reuse值是False;再次调用variable_scope时，传入reuse的值就会自动变为True**

### 演示作用域与操作符的受限范围——with tf.variable_scope("scope1") as sp
演示variable_scope的as用法，以及对应的作用域。
variable_scope还可以使用with variable_scope("name") as xxxscope的方式定义作用域，当使用这种方式时，所定义的作用域变量xxxscope将不再受到外围scope所限制，就是继承失效。

In [14]:
import tensorflow as tf
tf.reset_default_graph()
with tf.variable_scope("scope1") as sp: # 
    var1 = tf.get_variable("v", [1])
print("sp:", sp.name)
print("var1:", var1.name)

with tf.variable_scope("scope2"):
    var2 = tf.get_variable("v", [1])
    
    with tf.variable_scope(sp) as sp1: # sp1在scope2下，但是输出仍是scope1，没有改变。
        var3 = tf.get_variable("v3", [1]) # 在sp1下面定义的var3名字是scope1/v3:0，表明sp没有受到外层的限制

print("sp1:", sp1.name)
print("var2:", var2.name)
print("var3:", var3.name)

sp: scope1
var1: scope1/v:0
sp1: scope1
var2: scope2/v:0
var3: scope1/v3:0


### 操作符作用域——tf.name_scope
操作符不仅受到tf.name_scope作用域的限制，同时也受到tf.variable_scope作用域的限制

In [15]:
import tensorflow as tf
tf.reset_default_graph()
with tf.variable_scope("scope"):
    with tf.name_scope("bar"):
        v = tf.get_variable("v", [1])
        x = 1.0 + v
print("v:", v.name)
print("x.op", x.op.name)
# 可以看出，虽然v和x都在scope的bar下面，但是v的命名只受到scope的限制，
# tf.name_scope只能限制op，不能限制变量的命名

v: scope/v:0
x.op scope/bar/add


在tf.name_scope函数中，还可以使用空字符将作用域返回到顶层。

# 图的基本操作
